In [1]:
# hide
# !pip install seirsplus
# !pip install geopandas
# !pip install geopatra
# !pip install -U plotly
# !pip install xlrd
import contextlib
import io
import json
import random
import sys
import warnings
from pathlib import Path
from typing import List, Union

import folium
import geopandas as gpd
import geopatra
import networkx
import pandas as pd
from branca.colormap import linear
from IPython.display import display
from IPython.utils import io
from ipywidgets import (
    HTML,
    FloatLogSlider,
    FloatSlider,
    GridBox,
    HBox,
    IntSlider,
    Label,
    Layout,
    Output,
    SelectionSlider,
    VBox,
    interactive,
)
from seirsplus.models import *

warnings.filterwarnings("ignore")

In [2]:
# hide
assam_travel_history = Path("AssamTravelHistory.xlsx").resolve()
xl = pd.ExcelFile(assam_travel_history)

In [3]:
# hide
def read_assam_excel_to_df(filename: str) -> pd.DataFrame:
    xl = pd.ExcelFile(filename)
    df_list = []
    for sheet_name in xl.sheet_names:
        district_df = xl.parse(sheet_name)
        district_df["district"] = sheet_name
        district_df.drop(columns=["S.No."], inplace=True)
        df_list.append(district_df)
    return pd.concat(df_list, sort=False)


df = read_assam_excel_to_df(assam_travel_history)

In [4]:
# hide
df["DateOfArrival"] = pd.to_datetime(df["Date of arrival"])
df["DateOfReturn"] = pd.to_datetime(df["Date of Receipt"])
df.drop(columns=["Date of arrival", "Date of Receipt"], inplace=True)

In [5]:
# hide
df_copy = df
df_copy["Inflow"] = 1
assam_traveller_count_df = df_copy.groupby("district").agg({"Inflow": "sum"})
assam_traveller_count_df.reset_index(inplace=True)

In [6]:
# hide
def clean_district_names(dname: str):
    input_to_output_mapping = {
        "Cacher": "Cachar",
        "Kamrup_M": "Kamrup Metropolitan",
        "Kamrup_R": "Kamrup",
        "KarbiAnlong": "Karbi Anglong",
        "Baksha": "Baksa",
    }
    return input_to_output_mapping.get(dname, dname)

In [7]:
# hide
assam_traveller_count_df["district"] = assam_traveller_count_df.district.apply(
    clean_district_names
)

In [8]:
# hide
assam_pop_web_raw = pd.read_html(
    "https://www.census2011.co.in/census/state/districtlist/assam.html"
)
assam_pop_web_raw = assam_pop_web_raw[0][["District", "Population"]]
assam_pop_df = assam_pop_web_raw[
    ~(assam_pop_web_raw["District"].apply(lambda x: len(x)) > 21)
]

In [9]:
# hide
assam_pop_df_rename = assam_pop_df.rename(columns={"District": "district"})

In [10]:
# hide
assam_df = pd.merge(
    assam_pop_df_rename, assam_traveller_count_df, on="district", how="left"
)
assam_df.to_csv("AssamDistrictInfo.csv", index=False)